<a href="https://colab.research.google.com/github/bothananth-vb/LIME-ML/blob/main/IEEE_TASK_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv -O diabetes.csv

--2024-10-16 17:47:17--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘diabetes.csv’

diabetes.csv        100%[===================>]  22.73K  --.-KB/s    in 0s      

2024-10-16 17:47:17 (88.1 MB/s) - ‘diabetes.csv’ saved [23278/23278]



In [12]:
!pip install lime

data.columns = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"]  # Replace with your actual column names


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from lime.lime_tabular import LimeTabularExplainer
import warnings

# Suppress specific warnings from sklearn
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')

# Load the dataset
data = pd.read_csv('diabetes.csv')
data.columns = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin",
                "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"]

# Split the data into features and target
X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print evaluation metrics
print("\nModel Evaluation:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")

# Create a LIME explainer
explainer = LimeTabularExplainer(X_train.values,
                                   feature_names=X_train.columns,
                                   class_names=['No Diabetes', 'Diabetes'],
                                   discretize_continuous=True)

# Explain multiple instances and print the explanations
num_instances_to_explain = 5
for i in range(num_instances_to_explain):
    instance_to_explain = X_test.iloc[[i]]  # Keep it as a DataFrame
    explanation = explainer.explain_instance(instance_to_explain.values[0], model.predict_proba)

    print(f"\nExplanation for Instance {i+1} (Actual class: {y_test.iloc[i]}):")
    explanation_list = explanation.as_list()

    print(f"{'Feature':<30} {'Contribution to Prediction':>30}")
    print("-" * 60)

    for feature, weight in explanation_list:
        print(f"{feature:<30} {weight:>30.4f}")

    # Show prediction probability for diabetes
    prediction_proba = model.predict_proba(instance_to_explain)[0][1]
    print(f"\nPrediction probability for Diabetes: {prediction_proba:.4f}")
    print("-" * 60)



Model Evaluation:
Accuracy:  0.7857
Precision: 0.7143
Recall:    0.7018
F1-score:  0.7080

Explanation for Instance 1 (Actual class: 0):
Feature                            Contribution to Prediction
------------------------------------------------------------
Glucose <= 99.00                                      -0.2032
Age > 41.00                                            0.0841
Insulin > 127.00                                       0.0519
32.00 < BMI <= 36.60                                   0.0442
SkinThickness > 32.00                                  0.0353
BloodPressure <= 62.00                                -0.0098
3.00 < Pregnancies <= 6.00                            -0.0070
0.38 < DiabetesPedigreeFunction <= 0.63                        -0.0024

Prediction probability for Diabetes: 0.3600
------------------------------------------------------------

Explanation for Instance 2 (Actual class: 0):
Feature                            Contribution to Prediction
-------------------